# <font class="markdown-google-sans" size=6>**Applio NoUI**</font>
> <font class="markdown-google-sans">A simple, high-quality voice conversion tool focused on ease of use and performance.</font>

<!--a href="https://discord.gg/urxFjYmYYh"><img src="https://img.shields.io/discord/1096877223765606521?style=for-the-badge&logo=discord&logoColor=7289DA&labelColor=white&color=7289DA&label=Support"></a-->
<a href="https://discord.gg/urxFjYmYYh"><img src="https://img.shields.io/badge/Support-gray?logo=Discord&style=for-the-badge&labelColor=black&logoColor=white"></a>&emsp;<a href="https://github.com/IAHispano/Applio"><img src="https://img.shields.io/badge/GitHub-gray?logo=GitHub&style=for-the-badge&labelColor=black&logoColor=white"></a>&emsp;<a href="https://github.com/IAHispano/Applio/blob/main/TERMS_OF_USE.md"><img src="https://img.shields.io/badge/Terms-gray?logo=ReadTheDocs&style=for-the-badge&labelColor=black&logoColor=white"></a>

---

## <font class="markdown-google-sans">🤝 **Acknowledgments**</font>

To all external collaborators for their special help in the following areas:
* Hina (Encryption method)
* Poopmaster (Extra section)
* Shirou (UV installer)
* Bruno5430 (AutoBackup code, general notebook maintenance)
* kubinka0505 (notebook revamp)

---

## <font class="markdown-google-sans">⚠️ **Disclaimer**</font>
> By using Applio, you agree to comply with ethical and legal standards, respect intellectual property and privacy rights, avoid harmful or prohibited uses, and accept full responsibility for any outcomes, while Applio disclaims liability and reserves the right to amend these terms.

# <font class="markdown-google-sans">**Install Applio**</font>
If the runtime restarts, re-run the installation steps.

In [ ]:
#@title ## 🌱 <font class="markdown-google-sans">Initialize variables</font>
import os
import torch
import warnings

Git_Repository = "IAHispano/Applio" #@param {type: "string"}
Git_Repository = Git_Repository.replace(os.sep, "/")
Use_Legacy_Backup_Directory_Names = True #@param {type: "boolean"}

#-=-=-=-#

Title = Git_Repository.split("/")[-1]

Path_Base = os.getcwd()
Path_Code = os.path.join(Path_Base, Title)
Path_Drive_Parent = os.path.join(Path_Base, "drive")
Path_Drive_Full = os.path.join(Path_Drive_Parent, "MyDrive")

Path_Name_Backup = ("" if Use_Legacy_Backup_Directory_Names else os.sep).join((Title, "Backup"))
Path_Name_RVC_Backup = "RVC_Backup"

Path_Full_Backup = os.path.join(Path_Drive_Full, Path_Name_Backup)
Path_Logs = os.path.join(Path_Base, Title, "logs")

GPU = torch.cuda.is_available()
GPU_Warning = 'Runtime type is not set to "GPU", many actions will not be possible'
GPU_Error = "GPU is unavailable"

#-=-=-=-#

def gather_patterns(directory: str, patterns: list):
	files = []

	for pattern in list(patterns):
		for file in Path(directory).glob(pattern.strip(".")):
			files.append(str(file.resolve()))

	return files

def find_latest_file_in_directory(directory: str, patterns: list):
	files = gather_patterns(directory, patterns)

	if files:
		return sorted(files, key = os.path.getmtime)[-1]
	else:
		return None

In [ ]:
#@title ## ⚙️ <font class="markdown-google-sans">Setup runtime environment</font>
import os
import shutil
import warnings
from multiprocessing import cpu_count

if not GPU:
	warnings.warn(GPU_Warning)

cwd = os.getcwd()
# os.chdir(Path_Base)

#-=-=-=-#

Reinstall = True #@param {type: "boolean"}

#-=-=-=-#

CPU_Cores = cpu_count()
Post_Process = False

os.environ["CPU_Cores"] = str(CPU_Cores)

#LOGS_PATH = "/content/Applio/logs"
#BACKUPS_PATH = "/content/drive/MyDrive/ApplioBackup"

#is os.path.exists(Path_Code):
#	shutil.rmtree(Path_Code)

!git config --global advice.detachedHead false

if Reinstall:
	shutil.rmtree(Path_Code, ignore_errors = True)

!git clone https://github.com/{Git_Repository} --branch 3.6.0 --single-branch
os.chdir(Path_Code)
print()

# Install older Python
if os.sys.version_info.minor > 11:
	print("Installing older Python...")
	!apt update -y
	!apt install -y python3.11 python3.11-distutils python3.11-dev portaudio19-dev 7zip
	!update-alternatives --install /usr/bin/python3 python3 /usr/bin/python3.11 2
	!update-alternatives --set python3 /usr/bin/python3.11
	os.sys.path.append("/usr/local/lib/python3.11/dist-packages")
	print()

print("Installing requirements...")
!curl -LsSf https://astral.sh/uv/install.sh | sh
!uv pip install -r requirements.txt --extra-index-url https://download.pytorch.org/whl/cu128 --index-strategy unsafe-best-match
!uv pip install jupyter-ui-poll py7zr
print()

!python "core.py" "prerequisites" \
	--models "True" \
	--pretraineds_hifigan "True"
print()

print("Finished installing requirements!")

In [ ]:
#@title ## 💽 <font class="markdown-google-sans">Mount Drive</font>
import os
import shutil
from google.colab import drive

try:
	drive.mount(Path_Drive_Parent, force_remount = True)
except Exception as e:
	print(f"Failed to mount drive due to {e.__class__.__name__}: {e}")

os.chdir(Path_Base)
cwd = os.getcwd()

# Migrate directories to match documentation
if os.path.ismount(Path_Drive_Parent):
	os.chdir(Path_Drive_Full)

	if all((
		not os.path.exists(Path_Name_Backup),
		os.path.exists(Path_Name_RVC_Backup)
	)):
		shutil.move(Path_Name_RVC_Backup, Path_Name_Backup)

os.chdir(cwd)

# ⚡ <font class="markdown-google-sans">**Inference**</font>


In [ ]:
#@title ### 🤝 Sync with Google Drive
#@markdown > Run this cell to automatically Save / Load models from your mounted drive.
#@markdown >
#@markdown > This will merge and link your backup folder from Google Drive to this notebook
import os
import shutil
import tempfile
import warnings
import subprocess
from pathlib import Path
from IPython.display import display, clear_output

if not GPU:
	warnings.warn(GPU_Warning)

#-=-=-=-#

non_bak_folders = "mute", "reference", "zips", "mute_spin", "mute_spin-v2"
non_bak_path = os.path.join(tempfile.gettempdir(), "Applio", "rvc", "logs")

def press_button(button):
	button.disabled = True

def get_date(path):
	from datetime import datetime
	return datetime.fromtimestamp(int(os.stat(path).st_mtime))

def get_size(path):
	# use du for consistency with original behavior
	result = subprocess.run(
		["du", "-shx", "--apparent-size", path],
		capture_output = True,
		text = True
	)
	return result.stdout.split("\t")[0] + "B"

def sync_folders(folder, backup):
	from time import sleep
	from ipywidgets import widgets
	from jupyter_ui_poll import ui_events

	local = widgets.VBox([
		widgets.Label(f"Local: {Path_Logs.removeprefix(os.getcwd())}/{os.path.basename(folder)}/"),
		widgets.Label(f"Size: {get_size(folder)}"),
		widgets.Label(f"Last modified: {get_date(folder)}")
	])

	remote = widgets.VBox([
		widgets.Label(f"Remote: {Path_Name_RVC_Backup.removeprefix(os.getcwd())}/{os.path.basename(backup)}/"),
		widgets.Label(f"Size: {get_size(backup)}"),
		widgets.Label(f"Last modified: {get_date(backup)}")
	])

	separator = widgets.VBox([
		widgets.Label("|||"),
		widgets.Label("|||"),
		widgets.Label("|||")
	])

	radio = widgets.RadioButtons(
		options = [
			"Save local model to drive",
			"Keep remote model"
		]
	)

	button = widgets.Button(
		description = "Sync",
		icon = "upload",
		tooltip = "Sync model"
	)

	button.on_click(press_button)

	clear_output()
	print(f"Your local model '{os.path.basename(folder)}' is in conflict with it's copy in Google Drive.")
	print("Please select which one you want to keep:")
	display(widgets.Box([local, separator, remote]))
	display(radio)
	display(button)

	with ui_events() as poll:
		while not button.disabled:
			poll(10)
			sleep(0.1)

	if radio.value.lower().startswith("save"):
		if os.path.exists(backup):
			shutil.rmtree(backup)
		shutil.move(folder, backup)
	elif radio.value.lower().startswith("keep"):
		if os.path.exists(folder):
			shutil.rmtree(folder)

if os.path.ismount(Path_Drive_Parent):
	os.makedirs(Path_Name_RVC_Backup, exist_ok = True)
	os.makedirs(non_bak_path, exist_ok = True)

	if not os.path.islink(Path_Logs):
		for folder_name in non_bak_folders:
			folder = os.path.join(Path_Logs, folder_name)
			backup = os.path.join(Path_Name_RVC_Backup, folder_name)

			os.makedirs(folder, exist_ok=True)

			try:
				shutil.move(folder, non_bak_path)
			except Exception:
				pass

			if os.path.exists(folder):
				shutil.rmtree(folder, ignore_errors=True)

			folder = os.path.join(non_bak_path, folder_name)

			if os.path.exists(backup):
				if os.path.realpath(backup) != os.path.realpath(folder):
					shutil.rmtree(backup)

			try:
				os.symlink(folder, backup)
			except FileExistsError:
				pass

		# pathlib used only for directory iteration
		for model in Path(Path_Logs).iterdir():
			if model.is_dir() and not model.is_symlink():
				if model.name == ".ipynb_checkpoints":
					continue

				backup = os.path.join(Path_Name_RVC_Backup, model.name)

				if os.path.isdir(backup):
					sync_folders(str(model), backup)
				else:
					if os.path.exists(backup):
						os.remove(backup)

					shutil.move(str(model), backup)

		shutil.rmtree(Path_Logs)
		os.symlink(Path_Name_RVC_Backup, Path_Logs)

		clear_output()
		print("✅ Models are synced!")

	else:
		if os.path.exists(Path_Logs):
			shutil.rmtree(Path_Logs)

		if os.path.lexists(Path_Logs):
			if os.path.islink(Path_Logs) or os.path.isfile(Path_Logs):
				os.remove(Path_Logs)
			else:
				shutil.rmtree(Path_Logs)

		os.symlink(Path_Name_RVC_Backup, Path_Logs)

		clear_output()
		print("✅ Models already synced!")
else:
	print("❌ Drive is not mounted, skipping model syncing")
	print("To sync your models, first mount your Google Drive and re-run this cell")

In [ ]:
#@title ### 📥 <font size="markdown-google-sans">Download Model</font>
#@markdown > Hugging Face or Google Drive
import os
import warnings

if not GPU:
	warnings.warn(GPU_Warning)

cwd = os.getcwd()
os.chdir(Path_Code)

#-=-=-=-#

#@markdown > Separateble by `,`.
Model_Link = ""  # @param {type:"string"}
Models_Links = Model_Link.split(",")

#-=-=-=-#

for Link in Models_Links:
	!python "core.py" "download" \
		--model_link "{Link}"

	if Link != Models_Links[-1]:
		print()

os.chdir(cwd)

In [ ]:
#@title ### ⚡ <font class="markdown-google-sans">Run Inference</font>
#@markdown <font color=red>**Requires GPU.**</font>
import os
import torch
import warnings
import soundfile as sf
from IPython.display import Audio, HTML, display

if not GPU:
	raise Exception(GPU_Error)

cwd = os.getcwd()
os.chdir(Path_Code)

#-=-=-=-#
#@markdown > Attempts to find the `.pth` file in the directory below.
#@markdown >
#@markdown > Model `.index` is also found automatically.
Model_Directory = "/content/Applio/logs/Darwin" #@param {type: "string"}
if os.path.isfile(Model_Directory):
	warnings.warn("Model directory is a file, re-set to its dirname")
	print()
	Model_Directory = os.path.dirname(Model_Directory)

Path_Model_Inference = find_latest_file_in_directory(Model_Directory, ["*.pth"])
if not Path_Model_Inference:
	raise Exception("Model not found")

Path_Index_Inference = find_latest_file_in_directory(Model_Directory, ["*.index"])
if not Path_Index_Inference:
	warnings.warn("Index not found")

try:
	torch.load(Path_Model_Inference, map_location = "cpu", weights_only = True)
except Exception as e:
	raise Exception("Invalid model")

#-=-=-=-#
#@markdown ---
#@markdown ### 💬 Inputs
#@markdown > Both can be files.
#@markdown >
#@markdown > <font color=gold>⚠️ **If `Path_Input` is directory then `Path_Output` has to be directory as well.**</font>

Path_Input = "/content/drive/MyDrive/RVC/Data/Inputs/1" #@param {type: "string"}
Path_Output = "/content/drive/MyDrive/RVC/Data/Outputs/1" #@param {type: "string"}

Path_Model_Inference = find_latest_file_in_directory(Model_Directory, ["*.pth"])
os.environ["INFER_Path_Model_Inference"] = str(Path_Model_Inference)

Path_Index_Inference = find_latest_file_in_directory(Model_Directory, ["*.index"])
os.environ["INFER_Path_Index_Inference"] = str(Path_Index_Inference)

#-=-=-=-#
#@markdown ---
#@markdown ### 🔊 Playback
Autoplay = "No" #@param ["Input", "Output", "Output after Input", "No"]

#-=-=-=-#

# test loading model
try:
	torch.load(Path_Model_Inference, map_location = "cpu", weights_only = True)
except Exception:
	raise Exception("Invalid model")

# prepare input/output
Files_Final = []

if os.path.isfile(Path_Input):
	Files_Final.append(Path_Input)
	Path_Output_Final = Path_Output
else:
	if os.path.isfile(Path_Output):
		raise Exception("Input is a directory but output is a file")

	os.makedirs(Path_Output, exist_ok=True)

	for File in Path(Path_Input).glob("*.*"):
		File = str(File.resolve())
		try:
			sf.SoundFile(File)
		except Exception:
			print(f"File {File} is not a valid audio file, skipping")
			continue
		Files_Final.append(File)

# remove previous outputs if any
shutil.rmtree(Path_Output, ignore_errors = True)
os.makedirs(Path_Output, exist_ok = True)

print(f"{'Using model:':<15} {Path_Model_Inference}")
print(f"{'Using index:':<15} {Path_Index_Inference}")
print()

# Run inference
for File in Files_Final:
	Path_Input_Final = File

	# Determine actual output file path
	if os.path.isdir(Path_Output):
		base_name = os.path.splitext(os.path.basename(File))[0]
		Path_Output_Final = os.path.join(Path_Output, f"{base_name}.{Export_Format.lower()}")
	else:
		Path_Output_Final = Path_Output

	os.environ["INFER_Path_Input_Final"] = Path_Input_Final
	os.environ["INFER_Path_Output_Final"] = Path_Output_Final

	#print("Processing:", Path_Input_Final)

	if Post_Process:
		!python "core.py" "infer" \
			--pitch "$INFER_Pitch" \
			--volume_envelope "$INFER_Volume_Envelope" \
			--index_rate "$INFER_Search_Feature_Ratio" \
			--protect "$INFER_Protect_Voiceless_Consonants" \
			--f0_autotune "$INFER_Autotune" \
			--f0_autotune_strength "$INFER_Autotune_Strength" \
			--f0_method "$INFER_Pitch_Extraction_Algorithm" \
			--input_path "$INFER_Path_Input_Final" \
			--output_path "$INFER_Path_Output_Final" \
			--pth_path "$INFER_Path_Model_Inference" \
			--index_path "$INFER_Path_Index_Inference" \
			--split_audio "$INFER_Split_Audio" \
			--clean_audio "$INFER_Clean_Audio" \
			--clean_strength "$INFER_Clean_Strength" \
			--export_format "$INFER_Export_Format" \
			--embedder_model "$INFER_Embedder_Model" \
			--embedder_model_custom "$INFER_Embedder_Model_Custom" \
			--formant_shift "$INFER_Formant_Shifting" \
			--formant_qfrency "$INFER_Formant_Quefrency" \
			--formant_timbre "$INFER_Formant_Timbre" \
			--post_process "$INFER_Post_Process" \
			--reverb "$INFER_Reverb" \
			--pitch_shift "$INFER_Pitch_Shift" \
			--limiter "$INFER_Limiter" \
			--gain "$INFER_Gain" \
			--distortion "$INFER_Distortion" \
			--chorus "$INFER_Chorus" \
			--bitcrush "$INFER_BitCrush" \
			--clipping "$INFER_Clipping" \
			--compressor "$INFER_Compressor" \
			--delay "$INFER_Delay" \
			--reverb_room_size "$INFER_Reverb_Room_Size" \
			--reverb_damping "$INFER_Reverb_Damping" \
			--reverb_wet_gain "$INFER_Reverb_Wet_Gain" \
			--reverb_dry_gain "$INFER_Reverb_Dry_Gain" \
			--reverb_width "$INFER_Reverb_Width" \
			--reverb_freeze_mode "$INFER_Reverb_Freeze_Mode" \
			--pitch_shift_semitones "$INFER_Pitch_Shift_Semitones" \
			--limiter_threshold "$INFER_Limiter_Threshold" \
			--limiter_release_time "$INFER_Limiter_Release_Time" \
			--gain_db "$INFER_Gain_dB" \
			--distortion_gain "$INFER_Distortion_Gain" \
			--chorus_rate "$INFER_Chorus_Rate" \
			--chorus_depth "$INFER_Chorus_Depth" \
			--chorus_center_delay "$INFER_Chorus_Center_Delay" \
			--chorus_feedback "$INFER_Chorus_Feedback" \
			--chorus_mix "$INFER_Chorus_Mix" \
			--bitcrush_bit_depth "$INFER_BitCrush_Bit_Depth" \
			--clipping_threshold "$INFER_Clipping_Threshold" \
			--compressor_threshold "$INFER_Compressor_Threshold" \
			--compressor_ratio "$INFER_Compressor_Ratio" \
			--compressor_attack "$INFER_Compressor_Attack" \
			--compressor_release "$INFER_Compressor_Release" \
			--delay_seconds "$INFER_Delay_Seconds" \
			--delay_feedback "$INFER_Delay_Feedback" \
			--delay_mix "$INFER_Delay_Mix"
	else:
		!python "core.py" "infer" \
			--pitch "$INFER_Pitch" \
			--volume_envelope "$INFER_Volume_Envelope" \
			--index_rate "$INFER_Search_Feature_Ratio" \
			--protect "$INFER_Protect_Voiceless_Consonants" \
			--f0_autotune "$INFER_Autotune" \
			--f0_autotune_strength "$INFER_Autotune_Strength" \
			--f0_method "$INFER_Pitch_Extraction_Algorithm" \
			--input_path "$INFER_Path_Input_Final" \
			--output_path "$INFER_Path_Output_Final" \
			--pth_path "$INFER_Path_Model_Inference" \
			--index_path "$INFER_Path_Index_Inference" \
			--split_audio "$INFER_Split_Audio" \
			--clean_audio "$INFER_Clean_Audio" \
			--clean_strength "$INFER_Clean_Strength" \
			--export_format "$INFER_Export_Format" \
			--embedder_model "$INFER_Embedder_Model" \
			--embedder_model_custom "$INFER_Embedder_Model_Custom" \
			--formant_shifting "$INFER_Formant_Shift" \
			--formant_qfrency "$INFER_Formant_Quefrency" \
			--formant_timbre "$INFER_Formant_Timbre" \
			--post_process "$INFER_Post_Process"

	if os.path.exists(Path_Output_Final):
		# read audio length in seconds
		input_duration = sf.info(Path_Input_Final).duration
		output_duration = sf.info(Path_Output_Final).duration
		gap = 0.25  # seconds between playbacks

		autoplay_lower = Autoplay.lower()

		if autoplay_lower == "input":
			display(Audio(Path_Input_Final, autoplay = True))
			display(Audio(Path_Output_Final, autoplay = False))
		elif autoplay_lower == "output":
			display(Audio(Path_Input_Final, autoplay = False))
			display(Audio(Path_Output_Final, autoplay = True))
		elif autoplay_lower == "output after input":
			# create Audio objects
			input_audio = Audio(Path_Input_Final, autoplay = True)
			output_audio = Audio(Path_Output_Final, autoplay = False)

			# use _repr_html_() to embed HTML
			input_html = input_audio._repr_html_()
			output_html = output_audio._repr_html_()

			# combine with JS for delayed output
			html = f"""
			<div>
				Input:<br>{input_html}<br>
				Output:<br>{output_html}
			</div>
			<script>
				var input = document.querySelectorAll('audio')[document.querySelectorAll('audio').length-2];
				var output = document.querySelectorAll('audio')[document.querySelectorAll('audio').length-1];
				input.onended = () => {{
					setTimeout(() => {{
						output.play();
					}}, {int(gap * 1000)});
				}};
			</script>
			"""
			display(HTML(html))
		else:
			# No autoplay
			display(Audio(Path_Input_Final, autoplay = False))
			display(Audio(Path_Output_Final, autoplay = False))
	else:
		print(f'"{Path_Output_Final}" does not exist!')

	if File != Files_Final[-1]:
		print()

os.chdir(cwd)

In [ ]:
#@title ### 🔮 <font class="markdown-google-sans">Enable post-processing effects for inference</font>
Post_Process = False #@param {type: "boolean"}
os.environ["INFER_Post_Process"] = str(Post_Process)

#@markdown ---
#@markdown ### Pitch Shift
#@markdown
Pitch_Shift = False #@param {type: "boolean"}
os.environ["INFER_Pitch_Shift"] = str(Pitch_Shift)

Pitch_Shift_Semitones = 0.0 #@param {type: "slider", min: -12.0, max: 12.0, step: 0.1}
os.environ["INFER_Pitch_Shift_Semitones"] = str(Pitch_Shift_Semitones)

#@markdown ## Dynamics

#@markdown ---
#@markdown ### Gain
Gain = False #@param {type: "boolean"}
os.environ["INFER_Gain"] = str(Gain)

#@markdown
Gain_dB = 0.0 #@param {type: "slider", min: -20.0, max: 20.0, step: 0.1}
os.environ["INFER_Gain_dB"] = str(Gain_dB)

#@markdown ---
#@markdown ### Distortion
Distortion = False #@param {type: "boolean"}
os.environ["INFER_Distortion"] = str(Distortion)

#@markdown
Distortion_Gain = 0.0 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Distortion_Gain"] = str(Distortion_Gain)

#@markdown ---
#@markdown ### Clipping
Clipping = False #@param {type: "boolean"}
os.environ["INFER_Clipping"] = str(Clipping)

#@markdown
Clipping_Threshold = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Clipping_Threshold"] = str(Clipping_Threshold)

#@markdown ---
#@markdown ### Limiter
Limiter = False #@param {type: "boolean"}
os.environ["INFER_Limiter"] = str(Limiter)

Limiter_Threshold = -1.0 #@param {type: "slider", min: -20.0, max: 0.0, step: 0.1}
os.environ["INFER_Limiter_Threshold"] = str(Limiter_Threshold)

Limiter_Release_Time = 0.05 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
os.environ["INFER_Limiter_Release_Time"] = str(Limiter_Release_Time)

#@markdown ---
#@markdown ### Compressor
Compressor = False #@param{type: "boolean"}
os.environ["INFER_Compressor"] = str(Compressor)

#@markdown
Compressor_Threshold = -20.0 #@param {type: "slider", min: -60.0, max: 0.0, step: 0.1}
os.environ["INFER_Compressor_Threshold"] = str(Compressor_Threshold)

Compressor_Ratio = 4.0 #@param {type: "slider", min: 1.0, max: 20.0, step: 0.1}
os.environ["INFER_Compressor_Ratio"] = str(Compressor_Ratio)

Compressor_Attack = 0.001 #@param {type: "slider", min: 0.0, max: 0.1, step: 0.001}
os.environ["INFER_Compressor_Attack"] = str(Compressor_Attack)

Compressor_Release = 0.1 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
os.environ["INFER_Compressor_Release"] = str(Compressor_Release)

#@markdown ## Effects

#@markdown ---
#@markdown ### Reverb
Reverb = False #@param {type: "boolean"}
os.environ["INFER_Reverb"] = str(Reverb)

#@markdown
Reverb_Room_Size = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Reverb_Room_Size"] = str(Reverb_Room_Size)

Reverb_Damping = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Reverb_Damping"] = str(Reverb_Damping)

Reverb_Wet_Gain = 0.0 #@param {type: "slider", min: -20.0, max: 20.0, step: 0.1}
os.environ["INFER_Reverb_Wet_Gain"] = str(Reverb_Wet_Gain)

Reverb_Dry_Gain = 0.0 #@param {type: "slider", min: -20.0, max: 20.0, step: 0.1}
os.environ["INFER_Reverb_Dry_Gain"] = str(Reverb_Dry_Gain)

Reverb_Width = 1.0 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Reverb_Width"] = str(Reverb_Width)

Reverb_Freeze_Mode = 0.0 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Reverb_Freeze_Mode"] = str(Reverb_Freeze_Mode)

#@markdown ---
#@markdown ### Delay
Delay = False #@param {type: "boolean"}
os.environ["INFER_Delay"] = str(Delay)

#@markdown
Delay_Seconds = 0.1 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.01}
os.environ["INFER_Delay_Seconds"] = str(Delay_Seconds)

Delay_Feedback = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Delay_Feedback"] = str(Delay_Feedback)

Delay_Mix = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Delay_Mix"] = str(Delay_Mix)

#@markdown ---
#@markdown ### Chorus
Chorus = False #@param {type: "boolean"}
os.environ["INFER_Chorus"] = str(Chorus)

#@markdown
Chorus_Rate = 1.5 #@param {type: "slider", min: 0.1, max: 10.0, step: 0.1}
os.environ["INFER_Chorus_Rate"] = str(Chorus_Rate)

Chorus_Depth = 0.1 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Chorus_Depth"] = str(Chorus_Depth)

Chorus_Center_Delay = 15.0 #@param {type: "slider", min: 0.0, max:50.0, step: 0.1}
os.environ["INFER_Chorus_Center_Delay"] = str(Chorus_Center_Delay)

Chorus_Feedback = 0.25 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Chorus_Feedback"] = str(Chorus_Feedback)

Chorus_Mix = 0.5 #@param {type: "slider", min: 0.0, max: 1.0, step: 0.1}
os.environ["INFER_Chorus_Mix"] = str(Chorus_Mix)

#@markdown ---
#@markdown ### BitCrush
BitCrush = False #@param {type: "boolean"}
os.environ["INFER_BitCrush"] = str(BitCrush)

#@markdown
BitCrush_Bit_Depth = 4 #@param {type: "slider", min: 1, max: 16, step: 1}
os.environ["INFER_BitCrush_Bit_Depth"] = str(BitCrush_Bit_Depth)

# <font class="markdown-google-sans">🧠 **Training**</font>

In [ ]:
#@title ### <font class="markdown-google-sans">01. Setup Model Parameters</font>
import warnings

if not GPU:
	warnings.warn(GPU_Warning)

Model_Name = "Darwin" #@param {type: "string"}
Sample_Rate = "48 000 Hz" #@param ["32 000 Hz", "40 000 Hz", "48 000 Hz"] {allow-input: false}
Sample_Rate_Display = Sample_Rate

Sample_Rate = int("".join((character for character in Sample_Rate if character.isdigit())))

os.environ["Model_Name"] = Model_Name
os.environ["Sample_Rate"] = str(Sample_Rate)

In [ ]:
#@title ### <font class="markdown-google-sans">02. Preprocess Dataset</font>
import os
import shutil
import warnings

if not GPU:
	warnings.warn(GPU_Warning)

cwd = os.getcwd()
os.chdir(Path_Code)

#-=-=-=-#

Path_Dataset = "/content/drive/MyDrive/RVC/Data/Sets/Darwin" #@param {type: "string"}

Cut_Preprocess = "Automatic" #@param ["Automatic", "Simple", "Skip"]
Chunk_Length = 3 #@param {type: "slider", min: 0.5, max: 5.0, step: 0.5}

Process_Effects = False #@param {type: "boolean"}

Noise_Reduction = False #@param {type: "boolean"}
Noise_Reduction_Strength = 0.7 #@param {type: "slider", min: 0.0, max: 1.0, step:0.1}

Overlap_Length = 0.3 #@param {type: "slider", min: 0, max: 0.5, step: 0.1}
Normalization_Mode = "None" #@param ["Pre", "Post", "None"]
Normalization_Mode = Normalization_Mode.lower()

#-=-=-=-#

os.environ["Path_Dataset"] = Path_Dataset
os.environ["Cut_Preprocess"] = Cut_Preprocess
os.environ["Chunk_Length"] = str(Chunk_Length)
os.environ["Overlap_Length"] = str(Overlap_Length)
os.environ["Process_Effects"] = str(Process_Effects)
os.environ["Noise_Reduction"] = str(Noise_Reduction)
os.environ["Noise_Reduction_Strength"] = str(Noise_Reduction_Strength)
os.environ["Normalization_Mode"] = Normalization_Mode

#-=-=-=-#

Path_Logs_Current_Model = os.path.join(Path_Logs, Model_Name)

if os.path.exists(Path_Logs_Current_Model):
	for file in os.listdir(Path_Logs_Current_Model):
		obj = os.path.join(Path_Logs_Current_Model, file)

		if os.path.isdir(obj) and "sliced_audios" in file.lower():
			shutil.rmtree(obj, ignore_errors = True)
		elif file.lower().startswith(("model_info.json")):
			os.remove(obj)

	print(f'Removed sliced audio directories from "logs{os.sep}{Model_Name}" folder')
	print()

!echo python "core.py" "preprocess" \
	--model_name "$Model_Name" \
	--dataset_path "$Path_Dataset" \
	--sample_rate "$Sample_Rate" \
	--cpu_cores "$CPU_Cores" \
	--cut_preprocess "$Cut_Preprocess" \
	--process_effects "$Process_Effects" \
	--noise_reduction "$Noise_Reduction" \
	--noise_reduction_strength "$Noise_Reduction_Strength" \
	--chunk_len "$Chunk_Length" \
	--overlap_len "$Overlap_Length" \
	--normalization_mode "$Normalization_Mode"

print()

!python "core.py" "preprocess" \
	--model_name "$Model_Name" \
	--dataset_path "$Path_Dataset" \
	--sample_rate "$Sample_Rate" \
	--cpu_cores "$CPU_Cores" \
	--cut_preprocess "$Cut_Preprocess" \
	--process_effects "$Process_Effects" \
	--noise_reduction "$Noise_Reduction" \
	--noise_reduction_strength "$Noise_Reduction_Strength" \
	--chunk_len "$Chunk_Length" \
	--overlap_len "$Overlap_Length" \
	--normalization_mode "$Normalization_Mode"

os.chdir(cwd)

In [ ]:
#@title ### <font class="markdown-google-sans">03. Extract Features</font>
import os
import warnings

if not GPU:
    warnings.warn(GPU_Warning)

cwd = os.getcwd()
os.chdir(Path_Code)

#-=-=-=-#

F0_Method = "RMVPE" # @param ["Crepe", "Crepe Tiny", "RMVPE"] {allow-input: false}
F0_Method = F0_Method.lower().replace(" ", "-")

Embedder_Model = "ContentVec" # @param ["ContentVec", "Spin v2", "Chinese HuBERT Base", "Japanese HuBERT Base", "Korean HuBERT Base", "Custom"] {allow-input: false}
Embedder_Model = Embedder_Model.lower().replace(" ", "-")
Embedder_Model_Custom = "" #@param {type: "string"}

Mutes_Amount = 2 #@param {type: "slider", min: 0, max: 10, step: 1}

#-=-=-=-#

if not os.path.exists(
    os.path.join(Path_Logs_Current_Model, "sliced_audios")
):
    raise Exception("No directory with sliced audios has been found, please run the feature extraction cell")

if os.path.exists(Path_Logs_Current_Model):
	for file in os.listdir(Path_Logs_Current_Model):
		obj = os.path.join(Path_Logs_Current_Model, file)

		if os.path.isdir(obj):
			if file.lower().startswith(("extracted", "f0")):
				shutil.rmtree(obj, ignore_errors = True)
		elif file.lower().startswith(("config.json", "filelist.txt")):
			os.remove(obj)

	print(f'Removed extraction directories from "logs{os.sep}{Model_Name}" folder')
	print()

#-=-=-=-#

!echo python "core.py" "extract" \
    --model_name "{Model_Name}" \
    --f0_method "{F0_Method}" \
    --sample_rate "{Sample_Rate}" \
    --cpu_cores "{CPU_Cores}" \
    --gpu "0" \
    --embedder_model "{Embedder_Model}" \
    --embedder_model_custom "{Embedder_Model_Custom}" \
    --include_mutes "{Mutes_Amount}"

print()

!python "core.py" "extract" \
    --model_name "{Model_Name}" \
    --f0_method "{F0_Method}" \
    --sample_rate "{Sample_Rate}" \
    --cpu_cores "{CPU_Cores}" \
    --gpu "0" \
    --embedder_model "{Embedder_Model}" \
    --embedder_model_custom "{Embedder_Model_Custom}" \
    --include_mutes "{Mutes_Amount}"

os.chdir(cwd)

In [ ]:
#@title ### <font class="markdown-google-sans">04. Generate Index file</font>
#@markdown > <font color=red>**GPU required.**</font>
import os

if not GPU:
	raise Exception(GPU_Error)

cwd = os.getcwd()
os.chdir(Path_Code)

#-=-=-=-#

Algorithm = "Auto" #@param ["Auto", "Faiss", "KMeans"] {allow-input: false}

#-=-=-=-#

if os.path.exists(Path_Logs_Current_Model):
	for file in os.listdir(Path_Logs_Current_Model):
		obj = os.path.join(Path_Logs_Current_Model, file)

		if os.path.isfile(obj) and os.path.splitext(file.lower())[-1] == ".index":
			print(f'Removed first found ".index" file inside "logs" directory ("{Model_Name}.index")')
			os.remove(obj)
			break

	print()

!python "core.py" "index" \
	--model_name "{Model_Name}" \
	--index_algorithm "{Algorithm}"

os.chdir(cwd)

In [ ]:
#@title ### <font class="markdown-google-sans">05. Download Pretraineds</font>
#@markdown > Downloads the `G`enerator/`D`enominator files and generates the paths needed for the `Start Training` section.
import os
import re
import requests
from tqdm import tqdm

#@markdown ---

#@markdown 🧠 Uses smart auto-fill mechanics:
#@markdown <details>
#@markdown 	<summary>Tries to find <code>G</code>/<code>D</code> automatically based on <code>Sample_Rate</code> if any of these fields are repository URL </summary>
#@markdown 	<ul>
#@markdown 		<li>From:
#@markdown 			<ul>
#@markdown 				<li><code>G</code>: <code>hf.co/user/repo</code></li>
#@markdown 				<li><code>D</code>: <code>&ensp;</code></li>
#@markdown 			</ul>
#@markdown 		</li>
#@markdown 		<li>To:
#@markdown 			<ul>
#@markdown 				<li><code>G</code>: <code>hf.co/user/repo/resolve/main/G_40k.pth</code></li>
#@markdown 				<li><code>D</code>: <code>hf.co/user/repo/resolve/main/D_40k.pth</code></li>
#@markdown 			</ul>
#@markdown 		</li>
#@markdown 		<br>Throws an error if any URL is not found.
#@markdown 	</ul>
#@markdown </details>
#@markdown <details>
#@markdown 	<summary>If one of links is empty, it attempts to retrieve </code>G</code>/</code>D</code> from each other </summary>
#@markdown 	<ul>
#@markdown 		<li>From:
#@markdown 			<ul>
#@markdown 				<li><code>G</code>: <code>&ensp;</code></li>
#@markdown 				<li><code>D</code>: <code>hf.co/user/repo/resolve/main/D_48k.pth</code></li>
#@markdown 			</ul>
#@markdown 		</li>
#@markdown 		<li>To:
#@markdown 			<ul>
#@markdown 				<li><code>G</code>: <code>hf.co/user/repo/resolve/main/G_48k.pth</code></li>
#@markdown 				<li><code>D</code>: <code>hf.co/user/repo/resolve/main/D_48k.pth</code></li>
#@markdown 			</ul>
#@markdown 		</li>
#@markdown 		<br>Throws an error if any URL is not found.
#@markdown 	</ul>
#@markdown </details>
#@markdown <details>
#@markdown 	<summary>If no valid sample rate is found in <code>G</code>/<code>D</code> file base names, it tries to parse the one based on established <code>Sample_Rate</code> variable</summary>
#@markdown 	<ul>
#@markdown 		<li>From:
#@markdown 			<ul>
#@markdown 				<li><code>G</code>: <code>hf.co/user/repo/resolve/main/G_32k.pth</code></li>
#@markdown 				<li><code>D</code>: <code>&ensp;</code></li>
#@markdown 			</ul>
#@markdown 		</li>
#@markdown 		<li>To:
#@markdown 			<ul>
#@markdown 				<li><code>G</code>: <code>hf.co/user/repo/resolve/main/G_40k.pth</code></li>
#@markdown 				<li><code>D</code>: <code>hf.co/user/repo/resolve/main/D_40k.pth</code></li>
#@markdown 			</ul>
#@markdown 		</li>
#@markdown 		<br>Throws an error if any URL is not found.
#@markdown 	</ul>
#@markdown </details>

#@markdown ---

G = "http://hf.co/repo/user" #@param {type: "string"}
D = "" #@param {type: "string"}

Directory_Pretraineds_Destination = "/content/Applio/rvc/models/pretraineds/custom"

#-=-=-=-#

#@markdown ---

#@markdown > Downloads the checkpoint again, or if it doesn't exist, downloads other one with highest samplerate found.
#@markdown >
#@markdown > <font color=gold>⚠️ **If different samplerate checkpoints will be downloaded, the preprocessing and feature extraction cells have to be re-ran.**</font>
Forced = False #@param {type: "boolean"}

#@markdown ---

Pattern_Prefixes = "{GD}_, {GD}, _{GD}" #@param {type: "string"}
Pattern_Samplerate = "32k, 32000, 32, 40k, 40000, 40, 48k, 48000, 48" #@param {type: "string"}

Pattern_Prefixes = [p.strip() for p in Pattern_Prefixes.split(",") if p]
Pattern_Samplerate = [p.strip() for p in Pattern_Samplerate.split(",") if p]

def samplerate_to_number(n: str) -> int:
	samplerate = n.lower().replace("k", "000")
	return int(samplerate)

Pattern_Samplerate = sorted(Pattern_Samplerate, key = samplerate_to_number, reverse = True)

def url_exists(url: str) -> bool:
	try:
		r = requests.get(
			url,
			stream = True,
			headers = {"Range": "bytes=0-0"},
			timeout = 10,
			allow_redirects = True
		)
		return r.status_code in (200, 206)
	except:
		return False

def extract_samplerate(text: str, patterns_number: list):
	patterns = "|".join(patterns_number)
	match = re.search(rf"({patterns})", text, re.IGNORECASE)
	return match.group(1) if match else None

def normalize_repo_url(url: str) -> str:
	return url.rstrip("/")

def build_candidates(
	repo_url: str, target: str,
	patterns_string: list, patterns_number: list
):
	base = normalize_repo_url(repo_url)
	candidates = []

	for sr in patterns_number:
		for prefix in patterns_string:
			name = f"{prefix.format(GD = target)}{sr}.pth"
			candidates.append(f"{base}/resolve/main/{name}")

	return candidates

def infer_from_repo(
	repo_url: str, target: str,
	patterns_string: list, patterns_number: list
) -> str:
	for candidate in build_candidates(repo_url, target, patterns_string, patterns_number):
		if url_exists(candidate):
			return candidate

	raise FileNotFoundError(
		f"No valid {target} URL found in repo using supported patterns."
	)

def infer_from_other(
	other_url: str, want: str,
	patterns_string: list, patterns_number: list
) -> str:
	# extract samplerate from the other URL
	samplerate = extract_samplerate(other_url, patterns_number)
	if not samplerate:
		samplerate = f"{int(Sample_Rate) // 1000}k"

	# build candidates replace the prefix only
	candidates = []
	for prefix in patterns_string:
		# remove old prefix and samplerate
		name = f"{prefix.format(GD = want)}{samplerate}.pth"
		candidate = re.sub(r"([^/]+\.pth)$", name, other_url)
		candidates.append(candidate)

	# pick the first one that exists
	for c in candidates:
		if url_exists(c):
			return c

	raise FileNotFoundError(f"Could not infer {want} from {other_url} using supported patterns.")

#-=-=-=-#
# Auto fill

if not all((G, D)):
	print("Sample rate:", Sample_Rate_Display)
	print()

if G and not G.endswith(".pth"):
	G = infer_from_repo(
		G, "G",
		Pattern_Prefixes, Pattern_Samplerate
	)

if D and not D.endswith(".pth"):
	D = infer_from_repo(
		D, "D",
		Pattern_Prefixes, Pattern_Samplerate
	)

if not G and D:
	G = infer_from_other(
		D, "G",
		Pattern_Prefixes, Pattern_Samplerate
	)

if not D and G:
	D = infer_from_other(
		G, "D",
		Pattern_Prefixes, Pattern_Samplerate
	)

#-=-=-=-#

def download(url: str, destination: str, label: str, forced: bool):
	if not url:
		return None

	os.makedirs(destination, exist_ok = True)
	filename = os.path.basename(url)
	dest_path = os.path.join(destination, filename)

	if not forced and os.path.exists(dest_path):
		print(f'"{label}" already exists.')
		return dest_path

	print(f'Downloading "{label}": {filename}...')

	response = requests.get(url, stream=True)
	response.raise_for_status()
	total_size = int(response.headers.get("content-length", 0))

	with open(dest_path, "wb") as f, tqdm(
		total = total_size,
		unit = "iB",
		unit_scale = True,
		unit_divisor = 1024
	) as bar:
		for chunk in response.iter_content(chunk_size = 8192):
			if chunk:
				bar.update(f.write(chunk))

	return dest_path

#-=-=-=-#

Path_G = download(G, Directory_Pretraineds_Destination, "G", Forced)
Path_D = download(D, Directory_Pretraineds_Destination, "D", Forced)

if Forced:
	print()
	print()
else:
	print()

print(f"G Path: {Path_G}" if Path_G else "No G model downloaded.")
print(f"D Path: {Path_D}" if Path_D else "No D model downloaded.")

In [ ]:
#@title ### 06. <font class="markdown-google-sans">Start</title>
import os
from pathlib import Path

cwd = os.getcwd()
os.chdir(Path_Code)

#-=-=-=-#

#@markdown #### ⚙️ Settings
Total_Epoch = 500  #@param {type: "integer"}
Batch_Size = 8  #@param {type: "slider", min: 1, max: 25, step: 0}
Vocoder = "HiFi-GAN" #@param ["HiFi-GAN", "RefineGAN"]
Launch_Tensorboard = False # @param{type: "boolean"}

#@markdown #### ❓ Optional
#@markdown In case you select custom pretrained, you will have to download the pretraineds and enter their paths.
Custom_Pretrain = True #@param {type: "boolean"}
G = "/content/Applio/rvc/models/pretraineds/custom/G_48k.pth" #@param {type: "string"}
D = "/content/Applio/rvc/models/pretraineds/custom/D_48k.pth" #@param {type: "string"}

#@markdown ### ➡️ Choose how many epochs your model will be stored
Save_Every_Epoch = 10 # @param {type: "slider", min: 1, max: 100, step: 0}
Save_Only_Latest = True # @param{type: "boolean"}
Save_Every_Weights = True # @param{type: "boolean"}

#@markdown ---

#@markdown #### Overtraining
Detector = True # @param {type: "boolean"}
Threshold = 50 # @param {type: "slider", min: 1, max: 100, step: 0}

#@markdown ---

Pretrained = True #@param{type: "boolean"}
Cleanup = False #@param{type: "boolean"}
Cache_Data_In_GPU = True #@param {type: "boolean"}
Checkpointing = False

#-=-=-=-#

if Launch_Tensorboard:
	%load_ext tensorboard
	%tensorboard --logdir logs --bind_all

!python "core.py" train \
	--model_name "{Model_Name}" \
	--save_every_epoch "{Save_Every_Epoch}" \
	--save_only_latest "{Save_Only_Latest}" \
	--save_every_weights "{Save_Every_Weights}" \
	--total_epoch "{Total_Epoch}" \
	--sample_rate "{Sample_Rate}" \
	--batch_size "{Batch_Size}" \
	--gpu 0 \
	--pretrained "{Pretrained}" \
	--custom_pretrained "{Custom_Pretrain}" \
	--g_pretrained_path "{G}" \
	--d_pretrained_path "{D}" \
	--overtraining_detector "{Detector}" \
	--overtraining_threshold "{Threshold}" \
	--cleanup "{Cleanup}" \
	--cache_data_in_gpu "{Cache_Data_In_GPU}" \
	--vocoder "{Vocoder}" \
	--checkpointing "{Checkpointing}"

#-=-=-=-#

print("\n" * 3)

Files_Current_Model_Root_CKPT = Path(Path_Logs_Current_Model).glob("*.pth")
Latest = max(Files_Current_Model_Root_CKPT, key = lambda p: p.stat().st_mtime, default = None)

if Latest:
	print("Probable output file:".upper())
	print(Latest.resolve())
else:
	print("No .pth file found.")

#-=-=-=-#

os.chdir(cwd)

## 🚢 <font class="markdown-google-sans">Export-related</font>

In [ ]:
#@title ### 📉 <font class="markdown-google-sans">Optimize model</font>
#@markdown > Recompresses a model archive for inference by removing unused files and repacking with max compression.
#@markdown >
#@markdown > <font color=gold>⚠️ **Redundant for `Training` export mode.**</font>
import os
import shutil
import zipfile
import tempfile
import subprocess
from pathlib import Path

#-=-=-=-#

Packer = "7-Zip" #@param ["ZipFile", "7-Zip"]
use_7z = Format.lower().startswith("7")

DEBUG = True

#-=-=-=-#

def debug(msg):
	if DEBUG:
		print(f"[DEBUG] {msg}")

# detect 7z binary (Colab + local safe)
PATH_PACKAGE_7Z = None

for name in ("7z", "7zz"):
	found = shutil.which(name)
	if found:
		PATH_PACKAGE_7Z = found
		break

debug(f"7z binary: {PATH_PACKAGE_7Z}")

# compressor
class ArchiveCompressor:
	def __init__(self, destination: str, directory: str):
		self.destination = destination
		self.directory = directory
		assert os.path.exists(directory)

	def SevenZip(self, binary: str):
		if not binary:
			return False

		debug("Using 7z CLI compression")

		try:
			subprocess.run(
				[binary, "a", "-tzip", "-mx=9", self.destination, self.directory],
				check = True,
				stdout=subprocess.DEVNULL,
				stderr=subprocess.DEVNULL,
			)
			return self.destination
		except subprocess.CalledProcessError as e:
			debug(f"7z failed: {e}")
			return False

	def Zip(self):
		debug("Using Python ZipFile fallback")

		base_dir = Path(self.directory)

		with zipfile.ZipFile(
			self.destination,
			"w",
			compression = zipfile.ZIP_DEFLATED,
			compresslevel = 9,
		) as zip_out:
			for file_path in base_dir.rglob("*"):
				if file_path.is_file():
					rel_path = file_path.relative_to(base_dir.parent)
					zip_out.write(file_path, arcname=rel_path)

		return self.destination

# optimize model
def optimize_model(path: str, package_7z: str | None):
	if not zipfile.is_zipfile(path):
		raise ValueError("Input file is not a ZIP archive")

	with tempfile.TemporaryDirectory() as tmpdir:
		size_in = os.path.getsize(path)
		debug(f"Original size: {size_in / 1024 / 1024:.2f} MB")

		with zipfile.ZipFile(path, "r") as zip_in:
			zip_in.extractall(tmpdir)

		# expect exactly one top-level directory
		root_items = os.listdir(tmpdir)
		if len(root_items) != 1:
			raise ValueError("Expected a single top-level directory")

		top_dir = os.path.join(tmpdir, root_items[0])
		tmp_output = os.path.join(tmpdir, "._compressed.zip")

		# cleanup rules
		byteorder_path = os.path.join(top_dir, "byteorder")
		if os.path.isfile(byteorder_path):
			debug("Removing byteorder")
			os.remove(byteorder_path)

		data_dir = os.path.join(top_dir, ".data")
		if os.path.isdir(data_dir):
			debug("Removing .data directory")
			shutil.rmtree(data_dir)

		version_path = os.path.join(top_dir, "version")
		if os.path.isfile(version_path):
			with open(version_path, "r") as vf:
				content = vf.read().rstrip("\n")
			with open(version_path, "w") as vf:
				vf.write(content)

		# compress
		archive_compressor = ArchiveCompressor(tmp_output, top_dir)

		success = False
		if use_7z and package_7z:
			success = archive_compressor.SevenZip(package_7z)

		if not success:
			print("[Fallback] 7z unavailable or failed, using ZipFile")
			archive_compressor.Zip()

		size_out = os.path.getsize(tmp_output)
		debug(f"Optimized size: {size_out / 1024 / 1024:.2f} MB")

		shutil.move(tmp_output, path)

	return path, size_in, size_out

#-=-=-=-#

if not Latest:
	warnings.warn("Model not found, ignoring")

optimized_path, before, after = optimize_model(
	Latest,
	PATH_PACKAGE_7Z if use_7z else None,
)

print("\n✅ Optimization complete")
print(f"{'📦 Before:':<10} {before / 1024 / 1024:>8.2f} MB")
print(f"{'📉 After:':<10} {after / 1024 / 1024:>8.2f} MB")
print(f"{'💾 Saved:':<10} {(before - after) / 1024 / 1024:>8.2f} MB")

In [ ]:
#@title ### 📤 <font style="markdown-google-sans">Export model</font>
#@markdown - Training: Bigger file size, can continue training
#@markdown - Inference: Smaller file size, only for model inference
import os
import shutil
import zipfile
import warnings
import subprocess
from google.colab import files

#-=-=-=-#

#@markdown > ⚠️ <font color=red>**`7Z` is unsupported in this notebook's inference.**</font>
Format = "ZIP" #@param ["ZIP", "7Z"]

#@markdown > Exporting for training is only recommended for use outside of Applio, if you plan to resume training later, use [Sync with Google Drive](#scrollTo=section_inference_syncdrive) cell instead.
Mode = "Inference" #@param ["Inference", "Training"]

#-=-=-=-#

DEBUG = True

def debug(msg):
	if DEBUG:
		print(f"[DEBUG] {msg}")

# Detect 7z availability
Format = Format.upper()
sevenzip_mode = "zip"

if Format == "7Z":
	debug("Requested format: 7Z")

	# Try py7zr
	try:
		import py7zr
		sevenzip_mode = "py7zr"
		debug("Using py7zr backend")
	except Exception as e:
		debug(f"py7zr not available: {e}")

		# Try CLI 7z (NON-BLOCKING CHECK)
		try:
			subprocess.run(
				["7z", "--help"],
				stdout = subprocess.DEVNULL,
				stderr = subprocess.DEVNULL,
				timeout = 5,
				check = False,
			)
			sevenzip_mode = "cli"
			debug("Using 7z CLI backend")
		except Exception as e:
			warnings.warn(f"7z CLI unavailable ({e}), falling back to ZIP")
			sevenzip_mode = "zip"

else:
	debug("Requested format: ZIP")

# Paths
Path_Export = os.path.join(Path_Drive_Full, Title)
if not Use_Legacy_Backup_Directory_Names:
	Path_Export = os.path.join(Path_Export, "Exported")

logs_folder = Path_Logs_Current_Model
if not os.path.isdir(Path_Logs_Current_Model):
	raise FileNotFoundError(f"{Model_Name} model folder not found")

model_root = os.path.dirname(Path_Logs_Current_Model)
model_dir = os.path.join(model_root, Model_Name)

ext = ".7z" if sevenzip_mode in ("py7zr", "cli") else ".zip"
archive_path = os.path.join(os.getcwd(), f"{Model_Name}{ext}")

debug(f"Archive path: {archive_path}")
debug(f"Compression backend: {sevenzip_mode}")

# Helpers
def pack_directory(src_dir, archive_file):
	debug(f"Packing directory: {src_dir}")

	if sevenzip_mode == "py7zr":
		with py7zr.SevenZipFile(archive_file, "w") as zf:
			zf.writeall(src_dir, arcname = os.path.basename(src_dir))

	elif sevenzip_mode == "cli":
		subprocess.check_call([
			"7z", "a", "-t7z", archive_file, src_dir
		])

	else:
		with zipfile.ZipFile(archive_file, "w", zipfile.ZIP_DEFLATED) as zf:
			for root, _, files in os.walk(src_dir):
				for name in files:
					full_path = os.path.join(root, name)
					arcname = os.path.relpath(full_path, os.path.dirname(src_dir))
					zf.write(full_path, arcname)

def pack_files(files, archive_file):
	debug(f"Packing files: {files}")

	if sevenzip_mode == "py7zr":
		with py7zr.SevenZipFile(archive_file, "w") as zf:
			for f in files:
				if os.path.isfile(f):
					zf.write(f, arcname = os.path.relpath(f, model_root))
	elif sevenzip_mode == "cli":
		subprocess.check_call(
			["7z", "a", "-mx=9", "-t7z", archive_file] +
			[f for f in files if os.path.isfile(f)]
		)

	else:
		with zipfile.ZipFile(archive_file, "w", zipfile.ZIP_DEFLATED) as zf:
			for f in files:
				if os.path.isfile(f):
					arcname = os.path.relpath(f, model_root)
					zf.write(f, arcname)

# Export logic
debug(f"Export mode: {Mode}")
if Mode.lower().startswith("train"):
	pack_directory(model_dir, archive_path)
else:
	weight_files = [
		os.path.join(model_dir, f)
		for f in os.listdir(model_dir)
		if f.startswith(f"{Model_Name}_")
		and f.endswith(".pth")
	]

	if not weight_files:
		raise FileNotFoundError("Model has no weight file, please finish training first")

	weight_files.sort(key = os.path.getmtime, reverse = True)
	weight_file = weight_files[0]
	index_file = os.path.join(model_dir, f"{Model_Name}.index")

	debug(f"Selected weight: {weight_file}")
	pack_files((weight_file, index_file), archive_path)

# Move to Drive or download
if os.path.ismount(Path_Drive_Parent):
	os.makedirs(Path_Export, exist_ok = True)
	final_path = os.path.join(Path_Export, os.path.basename(archive_path))
	shutil.move(archive_path, final_path)
	print(f"Exported model to {final_path}")
else:
	print(f"Drive not mounted, downloading {os.path.basename(archive_path)}")
	files.download(archive_path)